# SAR 
* cooccurrence : 성능 그나마 아주 아주 아주 쪼끔 나옴 
* jaccard, lift : 성능 0...ㅎ

## 1. Load Data

In [3]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', 25)

df = pd.read_csv(os.path.join("..","..","data","TA_User_Reviws_Korea_all_new_df.csv")
df.head()

,userID,rating,location.locationId,location.name,location.placeType,location.reviewSummary.count,location.reviewSummary.rating,location.reviewSummary.locationId,tripInfo.stayDate,userProfile.hometown.location.name
0,0,4,306130,Lotte Hotel Seoul,ACCOMMODATION,4033.0,4.5,306130.0,2016-02-29,Jakarta
1,0,2,6352819,VIP TRAVEL,ATTRACTION,276.0,4.5,6352819.0,2016-02-29,Jakarta
2,1,5,9033360,Haagen Dazs,EATERY,10.0,4.5,9033360.0,2015-08-31,Seoul
3,1,5,9017499,Gongcha,EATERY,8.0,4.0,9017499.0,2015-11-30,Seoul
4,1,5,4076062,Mr. Pizza Terminal,EATERY,7.0,4.0,4076062.0,2015-06-30,Seoul


In [5]:
COLUMNS = ['userID', 'location.locationId', 'rating', 'tripInfo.stayDate']
df = df.loc[:,COLUMNS]
df.head()

,userID,location.locationId,rating,tripInfo.stayDate
0,0,306130,4,2016-02-29
1,0,6352819,2,2016-02-29
2,1,9033360,5,2015-08-31
3,1,9017499,5,2015-11-30
4,1,4076062,5,2015-06-30


## 2. git
* https://github.com/microsoft/recommenders/tree/master/reco_utils/recommender/sar 

In [6]:
pip install git+https://github.com/microsoft/recommenders 

  Cloning https://github.com/microsoft/recommenders to /tmp/pip-req-build-kn331z8d
  Running command git clone -q https://github.com/microsoft/recommenders /tmp/pip-req-build-kn331z8d
  Created wheel for reco-utils: filename=reco_utils-2019.9-cp36-none-any.whl size=164194 sha256=a416450a2c0177d071787ed4e7983296c571ede72191b3cb804a18e50ea95a10
  Stored in directory: /tmp/pip-ephem-wheel-cache-0kmoj8zx/wheels/b4/1c/70/c3f6005781b2b5a3370ec55bdd75c10ed1b07456bdfb1a0524
Successfully built reco-utils


In [8]:
!pip install papermill

     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 71kB 5.6MB/s 
     |████████████████████████████████| 747kB 8.5MB/s 
ERROR: nbclient 0.3.1 has requirement jupyter-client>=6.1.0, but you'll have jupyter-client 5.3.4 which is incompatible.


In [0]:
import sys
sys.path.append("../../")

import itertools
import logging
import os

import papermill as pm

from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

In [0]:
import datetime
import time 

## 3. Data Preprocessing 

In [0]:
# top k items to recommend
TOP_K = 10
DATA_SIZE = df.shape[0]

In [0]:
df["tripInfo.stayDate"] = df.apply(lambda x : time.mktime(datetime.datetime.strptime(x["tripInfo.stayDate"], "%Y-%m-%d").timetuple()), axis=1)

In [26]:
df.head()

,userID,location.locationId,rating,tripInfo.stayDate
0,0,306130,4,1.456704e+09
1,0,6352819,2,1.456704e+09
2,1,9033360,5,1.440979e+09
3,1,9017499,5,1.448842e+09
4,1,4076062,5,1.435622e+09


In [0]:
header = {
    "col_user": "userID",
    "col_item": "location.locationId",
    "col_rating": "rating",
    "col_timestamp": "tripInfo.stayDate",
}

In [0]:
train, test = python_stratified_split(df, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42)

In [30]:
print(train.shape, test.shape)

(22167, 4) (6394, 4)


## 4. Modeling 

In [0]:
model = SARSingleNode(
    similarity_type="cooccurrence", # jaccard, lift, cooccurrence 
    time_decay_coefficient=100, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [0]:
model.fit(train)

In [0]:
top_k = model.recommend_k_items(test, remove_seen=True)

In [61]:
display(top_k.head(20))

,userID,location.locationId,prediction
0,1,1169465,0.001202
1,1,553546,0.001172
2,1,324888,0.001117
3,1,2194168,0.001058
4,1,301815,0.000873
5,1,301253,0.000768
6,1,3477158,0.000735
7,1,592506,0.000723
8,1,306139,0.000718
9,1,1379963,0.000648


In [62]:
top_k.loc[top_k["prediction"]!=0] ## ㅋㅋmuzi...? 

,userID,location.locationId,prediction
0,1,1169465,0.001202
1,1,553546,0.001172
2,1,324888,0.001117
3,1,2194168,0.001058
4,1,301815,0.000873
...,...,...,...
17975,4552,308007,0.013322
17976,4552,1643534,0.013322
17977,4552,17678712,0.013322
17978,4552,3687319,0.013322


## 5. Evaluate the Result

In [0]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(col_user='userID', 
              col_item='location.locationId', 
              col_rating='rating', 
              col_prediction='prediction', 
              relevancy_method='top_k', 
              k=TOP_K)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

In [64]:
print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		 10
MAP:		 0.101074
NDCG:		 0.146962
Precision@K:	 0.044661
Recall@K:	 0.211642
